In [1]:
from __future__ import print_function
from vowpalwabbit import pyvw

In [2]:
tag = {}
tag['CC'] = 1
tag['CD'] = 2
tag['DT'] = 3
tag['EX'] = 3
tag['FW'] = 4
tag['IN'] = 5
tag['JJ'] = 6
tag['JJR'] = 6
tag['JJS'] = 6
tag['MD'] = 7
tag['NN'] = 8
tag['NNP'] = 8
tag['NNPS'] = 8
tag['NNS'] = 8
tag['PDT'] = 3
tag['POS'] = 11
tag['PRP'] = 9
tag['PRP$'] = 9
tag['RB'] = 10
tag['RBR'] = 10
tag['RBS'] = 10
tag['RP'] = 11
tag['SYM'] = 12
tag['TO'] = 11
tag['UH'] = 13
tag['VB'] = 14
tag['VBD'] = 14
tag['VBG'] = 14
tag['VBN'] = 14
tag['VBP'] = 14
tag['VBZ'] = 14
tag['WDT'] = 3
tag['WP'] = 9
tag['WP$'] = 9
tag['WRB'] = 10

In [3]:
s = ''
punc = ['#', '$',"''",'(',')',',','.',':',"``"]
sentence = []
sentences = []
pos_tags = set([])
file = open('train.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences.append(sentence)
        sentence = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence.append(tup)

In [13]:
sentence_test = []
sentences_test = []
pos_tags_test = set([])
file = open('test.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences_test.append(sentence_test)
        sentence_test = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags_test.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence_test.append(tup)

In [ ]:
sentences[0][1][0]

In [ ]:
class SequenceLabeler(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        # you must must must initialize the parent class
        # this will automatically store self.sch <- sch, self.vw <- vw
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        
        # set whatever options you want
        sch.set_options( sch.AUTO_HAMMING_LOSS | sch.AUTO_CONDITION_FEATURES )

    def _run(self, sentence):   # it's called _run to remind you that you shouldn't call it directly!
        output = []
        for n in range(len(sentence)):
            pos,word = sentence[n]
            # use "with...as..." to guarantee that the example is finished properly
            with self.vw.example({'w': [word]}) as ex:
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'), (n-1, 'q')])
                output.append(pred)
        return output

## Customizing features


In [40]:
class SequenceLabeler2(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_HAMMING_LOSS)

    def _run(self, sentence):
        output = []
        #loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<s>'
            prev_n_2 = output[n-2] if n > 1 else '<s>'
            posnew, prevWord = sentence[n-1] if n> 0 else ('<s>','<s>')
            with self.vw.example({'w': [word],'p': [prevPred], 'r':[prevWord]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n,'r')])
                output.append(pred)
                
        return output
    
#sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)
#sequenceLabeler2.learn(my_dataset)
#print(sequenceLabeler2.predict( [(0,w) for w in "the sandwich ate a monster".split()] ))

## Customizing loss functions

In [47]:
wt = {}

wt[1] = 1.0 
wt[2] = 1.0
wt[3] = 1.0
wt[4] = 1.0
wt[5] = 1.0
wt[6] = 1.1
wt[7] = 1.0
wt[8] = 1.0
wt[9] = 1.0
wt[10] = 1.1
wt[11] = 1.0
wt[12] = 1.0
wt[13] = 1.0
wt[14] = 1.1

In [72]:
class SequenceLabeler3(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_CONDITION_FEATURES)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            with self.vw.example({'w': [word]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n-1,'q')])
                output.append(pred)
                if pred != pos:
                    loss += wt_err[pos]            
        self.sch.loss(loss)
                
        return output

In [110]:
def calculate_loss():
    print("Calculating loss...")
    loss_t=[]
    s=0
    for sen in sentences_test:
        s+=1
        if(s%1000==0):
            print(s)
        temp=[]
        for w in sen:
            temp.append((1,w[1]))
        out = sequenceLabeler3.predict(temp)
        #print(out)
        loss=0
        c=0
        for i in out:        
            if sen[c][0] in tag_count:
                tag_count[sen[c][0]] += 1
            else:
                tag_count[sen[c][0]] = float(1)
                tag_error[sen[c][0]] = float(0)

            if i != sen[c][0]:
                loss+=1
                tag_error[sen[c][0]] += 1
                #print ("Predicted value at ",c," is ",i)
                #print (sentences[0][c])
            c+=1
        loss_t.append(loss/float(c))


    print (sum(loss_t) / float(len(loss_t)))

In [111]:
import operator
def updateWeights():
    print("Updating weights...")
    tag_loss = {}
    for t in tag_count:
        tag_loss[t] = tag_error[t]/tag_count[t]

    sorted_loss = sorted(tag_loss.items(), key=operator.itemgetter(1))

    for tp in sorted_loss:
            print( tp[0], " = ", tag_error[tp[0]], "/", tag_count[tp[0]], " = ", tp[1] )
            wt_err[tp[0]]=tp[1]
            
    avg_err=sum(wt_err)/14.0
    wt_err[:]=[1+(x - avg_err) for x in wt_err]
    
        
        
        

In [116]:
vw = pyvw.vw("--search 14 --audit --quiet --search_task hook --ring_size 1024 -f pos_tagger.model")

In [117]:
sequenceLabeler3 = vw.init_search_task(SequenceLabeler3)

In [118]:
wt_err=[1]*15
tag_count = {}
tag_error = {}

In [119]:
for i in range(4):
    print (i)
    sequenceLabeler3.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    calculate_loss()
    updateWeights()

0
Calculating loss...
1000
2000
0.136344372717
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  21.0 / 1624.0  =  0.0129310344828
5  =  94.0 / 5071.0  =  0.0185367777559
7  =  9.0 / 470.0  =  0.0191489361702
9  =  46.0 / 1349.0  =  0.0340993328391
3  =  163.0 / 4280.0  =  0.0380841121495
8  =  1377.0 / 14612.0  =  0.0942376129209
14  =  1299.0 / 6232.0  =  0.208440308087
10  =  444.0 / 1567.0  =  0.283343969368
6  =  1242.0 / 3243.0  =  0.382978723404
2  =  755.0 / 1918.0  =  0.393639207508
4  =  3.0 / 4.0  =  0.75
13  =  2.0 / 2.0  =  1.0
1
Calculating loss...
1000
2000
0.138028463875
Updating weights...
11  =  12.0 / 1624.0  =  0.00738916256158
1  =  11.0 / 1214.0  =  0.00906095551895
5  =  70.0 / 5071.0  =  0.0138039834352
7  =  7.0 / 470.0  =  0.0148936170213
9  =  30.0 / 1349.0  =  0.0222386953299
3  =  172.0 / 4280.0  =  0.0401869158879
8  =  1562.0 / 14612.0  =  0.106898439639
14  =  1432.0 / 6232.0  =  0.229781771502
4  =  1.0 / 4.0  =  0.25
10  =  424.0 / 15

In [ ]:
temp=[]
for w in sentences[0]:
    temp.append((0,w[1]))
print (temp)

In [ ]:
out = sequenceLabeler2.predict(temp)
print(out)
loss=0
c=0
for i in out:
    if i != sentences[0][c][0]:
        loss+=1
        print ("Predicted value at ",c," is ",i)
        print (sentences[0][c])
    c+=1



In [ ]:
sentences_test[0]